<a href="https://colab.research.google.com/github/dnyanda12/assignments/blob/main/ML_a1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap.geemap as map
import ee
ee.Authenticate()
ee.Initialize(project='ee-dnyandagawathe24')
m = map.Map()

In [4]:
Guwahati = ee.FeatureCollection('projects/ee-dnyandagawathe24/assets/GUWAHATI')
m.addLayer(Guwahati, {}, 'Guwahati')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
vis_para = {
    'min': 0,
    'max': 100,
    'palette': ['#006400', '#ffbb22', '#ffff4c', '#f096ff', '#fa0000', '#b4b4b4', '#f0f0f0', '#0064c8', '#0096a0', '#00cf75', '#fae6a0']}

In [6]:
legend = {'Treecover': '#006400',
          'Shrubland': '#ffbb22',
          'Grassland': '#ffff4c',
          'Croplands': '#f096ff',
          'Built-up': '#fa0000',
          'Bare/Sparse vegetation': '#b4b4b4',
          'Snow-ice': '#f0f0f0',
          'Permanent water bodies': '#0064c8',
          'Herbaceous wetland': '#0096a0',
          'Mangroves': '#00cf75',
          'Moss and Lichen': '#fae6a0'
          }

In [7]:
Guwahati_esa =ee.ImageCollection('ESA/WorldCover/v200').mean().clip(Guwahati).select('Map')
m.addLayer(Guwahati_esa, vis_para, 'ESA WorldCover');
m.add_legend(Title = "Legend", legend_dict=legend,**vis_para)

In [9]:
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterBounds(Guwahati).filterDate('2022-01-01','2023-01-01').filterMetadata('CLOUD_COVER', 'less_than', 5).mean().clip(Guwahati) # Changed 'cli' to 'clip' to clip the image to the Guwahati boundary.

In [11]:
TrainingP = Guwahati_esa.sample(**{
    'region': Guwahati,
    'scale': 20,
    'numPixels': 4000,
    'seed': 1,
    'geometries': True
})
m.addLayer(TrainingP, {}, 'Training Points')

In [12]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6']
label = 'Map'
training = landsat8.select(bands).sampleRegions(**{
    'collection': TrainingP,
    'properties': [label],
    'scale': 20
})
trained = ee.Classifier.smileRandomForest(10).train(training, label, bands)
result = landsat8.select(bands).classify(trained)

In [13]:
m.addLayer(result, vis_para, 'Supervised Classification')
m.addLayerControl(position = 'bottomleft')
m

Map(bottom=55846.0, center=[26.44598399885638, 92.30438232421876], controls=(WidgetControl(options=['position'…